In [ ]:
#### U is filled with random values
#### Shape is embed_matrix.shape[0] ,n_users
#### Implement hinge loss to find the prob P(S|user) as equation given in paper
#### user embeddings are learnt during the above loss computation
#### Use file pickled from create_user_train to get w_l per user

In [5]:



# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os, sys
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
#import loss_func as tf_func

import pickle
from collections import namedtuple




Word2Vec = namedtuple('Word2Vec', ['train_inputs', 'train_labels', 'loss', 'optimizer', 'global_step',
                                    'embeddings', 'normalized_embeddings', 'valid_embeddings','similarity',
                                    'saver','summary', 'summary_writer'])




In [6]:
def build_model(sess, graph, u_embed_rows):
    """
    Builds a tensor graph model
    """
    model = None
    with graph.as_default():
    # Ops and variables pinned to the CPU because of missing GPU implementation
        with tf.device('/cpu:0'):
          # Input data.
#             train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
#             train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
#             valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

            global_step = tf.Variable(0, trainable=False)

            # Look up embeddings for inputs.
            embeddings = tf.Variable(tf.random_uniform([u_embe_rows,n_users ], -1.0, 1.0))
            #embed = tf.nn.embedding_lookup(embeddings, train_inputs)

            sm_weights = tf.Variable(
              tf.truncated_normal([vocabulary_size, embedding_size],
                                  stddev=1.0 / math.sqrt(embedding_size)))

            # Get context embeddings from lables
            true_w = tf.nn.embedding_lookup(sm_weights, train_labels)

            true_w = tf.reshape(true_w, [-1, embedding_size])

            # Construct the variables for the NCE loss
            nce_weights = tf.Variable(
              tf.truncated_normal([vocabulary_size, embedding_size],
                                  stddev=1.0 / math.sqrt(embedding_size)))
            nce_biases = tf.Variable(tf.zeros([vocabulary_size]))




    
    sample = np.random.choice(vocabulary_size, num_sampled, p=unigram_prob, replace=False)

    loss = -tf.reduce_mean(tf_func.nce_loss(embed, nce_weights, nce_biases, train_labels, sample, unigram_prob))

    # tf.summary.scalar('loss', loss)

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(.05).minimize(loss, global_step=global_step)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm

    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)

    saver = tf.train.Saver(tf.global_variables())

    # Save summary
    # summary = tf.summary.merge_all()
    # summary_writer = tf.summary.FileWriter(summary_path + '/summary', sess.graph)
    summary = None
    summary_writer = None

    tf.global_variables_initializer().run()
    print("Initialized")

    model = Word2Vec(train_inputs, train_labels, loss, optimizer, global_step, embeddings,
                    normalized_embeddings, valid_embeddings, similarity, saver, summary, summary_writer)


    return model


In [15]:
def train(sess, model, data, dictionary, batch_size, num_skips, skip_window,
          max_num_steps, checkpoint_step, loss_model):

    average_loss_step = max(checkpoint_step/10, 100)

    average_loss = 0
    for step in xrange(max_num_steps):
        batch_inputs, batch_labels = generate_batch(data, batch_size, num_skips, skip_window)
        feed_dict = {model.train_inputs.name: batch_inputs, model.train_labels.name: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        #print("------got first feed_dict ---------")
        #print(feed_dict)
        _, loss_val = sess.run([model.optimizer, model.loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % average_loss_step == 0:
            if step > 0:
                average_loss /= average_loss_step
          # The average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0
          # model.summary_writer.add_summary(summary, model.global_step.eval())
          # model.summary_writer.flush()

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % checkpoint_step == 0:
            sim = model.similarity.eval()
        for i in xrange(valid_size):
            valid_word = reverse_dictionary[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            nearest = (-sim[i, :]).argsort()[1:top_k + 1]
            log_str = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
            close_word = reverse_dictionary[nearest[k]]
            log_str = "%s %s," % (log_str, close_word)
        print(log_str)
      #chkpt_path = os.path.join(checkpoint_model_path, 'w2v_%s.cpkt'%(loss_model))
      #model.saver.save(sess, chkpt_path, global_step=model.global_step.eval())


    # model.summary_writer.close()

    # Saving the final embedding to a file
    final_embeddings = model.normalized_embeddings.eval()

    return final_embeddings


In [14]:
if __name__ == '__main__':
    
    # maximum training step
    
    embed_matrix, unigram_prob, wrd2idx, word_counter,n_users = pickle.load(open('train_embeddings.pkl', 'rb'))
    print(n_users)
    graph = tf.Graph()
    with tf.Session(graph=graph) as sess:

        ####################################################################################
        # Step 4: Build and train a skip-gram model.

        model = build_model(sess, graph, embed_matrix.shape[0], n_users)
        
        user_embeddings = train(sess, model, data, dictionary, batch_size, num_skips, skip_window,
                            max_num_steps, checkpoint_step, loss_model)


        ####################################################################################
        # Step 7: Save the trained model.
        trained_steps = model.global_step.eval()

        maybe_create_path(model_path)
        model_filepath = os.path.join(model_path, 'word2vec_%s.model'%(loss_model))
        print("Saving word2vec model as [%s]"%(model_filepath))
        k=str(model_filepath)
        pickle.dump([dictionary, trained_steps, embeddings], open(k, 'wb'))


TypeError: build_model() missing 1 required positional argument: 'loss_model'